# Trio Variant Analysis Pipeline
Extracting analysis-ready variants for a father/mother/child trio from the 1000 genomes dataset.


In [17]:
import sys
from os.path import join

# Add the root of the repo to the current path
sys.path.append('..')

In [18]:
# Import Python helpers
from src.gatk.curate_genotype_dataset import CurateGenotypeDataset
from src.gatk.extract_sample_variants import ExtractGenomicVariants
from src.hail.utils import vcf_path_to_mt, hl

In [19]:
# Use inline plotting
%matplotlib inline


# Define variables for the pipeline
### Reads
Reads are obtained from the 1000 genomes FTP site for a trio of individuals (father/mother/child). For the example pipeline we restrict our analysis to chromosome 20 for each individual, although this can be easily adjusted by modifying the `intervals` variable of CurateGenotypeDataset.

The trio define the cohort of samples we will use for this analysis.

### Reference
The B37 reference genome from the Broad Institute is used as the refererence genome, since the phase3 data of the 1kg project was aligned against GRCh37. See https://www.internationalgenome.org/data-portal/data-collection/phase-3 for details.


### Known Sites for Variant Identification
The known sites of variation are obtained from the Broad Institutes reference bundle for b37. Here we only use the DBSNP resource.

### Known Sites VQSR
The known sites of variation are obtained from the Broad Institutes reference bundle for b37, and are used for VQSR's training and test set. The resources chosen are the same used in the VariantRecalibrator example as in
https://gatk.broadinstitute.org/hc/en-us/articles/360036510892-VariantRecalibrator.


In [20]:
PHASE_3_1KG_ROOT_FTP = "ftp://ftp.1000genomes.ebi.ac.uk/vol1/ftp/phase3/data"


# Reads for father/mother/child
READS_BAM_URIS=[
    # Father
    join(PHASE_3_1KG_ROOT_FTP, "HG00731/alignment/HG00731.chrom20.ILLUMINA.bwa.PUR.low_coverage.20130422.bam"),
    # Mother
    join(PHASE_3_1KG_ROOT_FTP, "HG00732/alignment/HG00732.chrom20.ILLUMINA.bwa.PUR.low_coverage.20130422.bam"),
    # Child
    join(PHASE_3_1KG_ROOT_FTP, "HG00733/alignment/HG00733.chrom20.ILLUMINA.bwa.PUR.low_coverage.20130415.bam"),
]

# Reference genome
REF_FA_GZ_URI = "ftp://ftp.1000genomes.ebi.ac.uk/vol1/ftp/technical/reference/human_g1k_v37.fa"

# Known sites for variant extraction
KNOWN_EXTRACTION_VCF_GZ_URIS = [
    "ftp://gsapubftp-anonymous@ftp.broadinstitute.org/bundle/b37/dbsnp_138.b37.vcf.gz"
]

# Known sites for VQSR model
KNOWN_SITES_VQSR_URIS = [
    "ftp://gsapubftp-anonymous@ftp.broadinstitute.org/bundle/b37/hapmap_3.3.b37.vcf.gz",
    "ftp://gsapubftp-anonymous@ftp.broadinstitute.org/bundle/b37/1000G_omni2.5.b37.vcf.gz",
    "ftp://gsapubftp-anonymous@ftp.broadinstitute.org/bundle/b37/1000G_phase1.snps.high_confidence.b37.vcf.gz",
    "ftp://gsapubftp-anonymous@ftp.broadinstitute.org/bundle/b37/dbsnp_138.b37.vcf.gz"
]

# Extract Genomic Variants

Below we perform the following steps:
    
1. Download all data
   1. Sample reads (i.e., a cohort of reads)
   2. Known sites
   3. Reference
2. Perform data preprocessing
   1. Filtering, sorting and indexing cohort reads
   2. Creating indices for known sites
   3. Creating an index for the reference
3. Perform ReadsPipelineSpark for each sample, obtaining analysis-ready variants for each sample of the cohort


In [21]:
variant_discovery = ExtractGenomicVariants(
    volume_hostpath='/data/Genomics',
    paired_reads_file_bam_uris=READS_BAM_URIS,
    ref_file_fasta_uri=REF_FA_GZ_URI,
    known_sites_vcf_uris=KNOWN_EXTRACTION_VCF_GZ_URIS,
    dry_run=False,
    spark_driver_memory= '5g',
    spark_executor_memory= '8g',
    java_options= '-Xmx48g',
)

# Run the pipeline to obtain the GVCF path for each sample
sample_gvcf_paths = variant_discovery.run_pipeline()


2022-05-26 00:26:35.357 | INFO     | src.gatk.base:download_uris:184 - Skipping download of URI ftp://ftp.1000genomes.ebi.ac.uk/vol1/ftp/phase3/data/HG00731/alignment/HG00731.chrom20.ILLUMINA.bwa.PUR.low_coverage.20130422.bam as file /data/Genomics/reads/human/HG00731.chrom20.ILLUMINA.bwa.PUR.low_coverage.20130422.bam already exists.
2022-05-26 00:26:35.358 | INFO     | src.gatk.base:download_uris:184 - Skipping download of URI ftp://ftp.1000genomes.ebi.ac.uk/vol1/ftp/phase3/data/HG00732/alignment/HG00732.chrom20.ILLUMINA.bwa.PUR.low_coverage.20130422.bam as file /data/Genomics/reads/human/HG00732.chrom20.ILLUMINA.bwa.PUR.low_coverage.20130422.bam already exists.
2022-05-26 00:26:35.358 | INFO     | src.gatk.base:download_uris:184 - Skipping download of URI ftp://ftp.1000genomes.ebi.ac.uk/vol1/ftp/phase3/data/HG00733/alignment/HG00733.chrom20.ILLUMINA.bwa.PUR.low_coverage.20130415.bam as file /data/Genomics/reads/human/HG00733.chrom20.ILLUMINA.bwa.PUR.low_coverage.20130415.bam already 

2022-05-26 00:26:35.382 | INFO     | src.gatk.base:_run_command_if_file_not_exists:46 - File /data/sample_gvcfs/human/HG00732.g.vcf already exists; skipping creation.
2022-05-26 00:26:35.383 | INFO     | src.gatk.base:_log_operation_header:62 - --------------------
2022-05-26 00:26:35.383 | INFO     | src.gatk.base:_log_operation_header:63 - Stage: Creating sample GVCF
2022-05-26 00:26:35.383 | INFO     | src.gatk.base:_log_operation_header:64 - --------------------
2022-05-26 00:26:35.384 | INFO     | src.gatk.extract_sample_variants:create_sample_gvcf:253 - Running command: gatk ReadsPipelineSpark --input /data/reads/human/HG00733.chrom20.ILLUMINA.bwa.PUR.low_coverage.20130415.paired.sorted.bam --reference /data/reference/human/human_g1k_v37.fa --known-sites /data/known_sites/human/dbsnp_138.b37.vcf --output /data/sample_gvcfs/human/HG00733.g.vcf -align --conf 'spark.driver.memory=5g' --conf 'spark.executor.memory=8g' --remove-all-duplicates true --java-options -Xmx48g --native-pair-

# Curate Genotype Dataset

Below we perform the following steps:

4. Combine cohort samples into a GenomicsDB
5. Perform joint genotyping on the cohort with HaplotypeCaller
6. Fit/applies a variant recalibration model to obtain variant quality scores
7. Applies VQSR to filter low quality variants

In [22]:
curated_genotyped_dataset = CurateGenotypeDataset(
    volume_hostpath='/data/Genomics',
    intervals=['20'],
    gvcf_paths=sample_gvcf_paths,
    annotations=[
        "QD",
        "MQ",
        "MQRankSum",
        "ReadPosRankSum",
        "FS",
        "SOR"
    ],
    known_sites_vqsr=KNOWN_SITES_VQSR_URIS,
    resources=[
        ('hapmap,known=false,training=true,truth=true,prior=15.0', 'hapmap_3.3.b37.vcf'),
        ('omni,known=false,training=true,truth=false,prior=12.0', '1000G_omni2.5.b37.vcf'),
        ('1000G,known=false,training=true,truth=false,prior=10.0', '1000G_phase1.snps.high_confidence.b37.vcf'),
        ('dbsnp,known=true,training=false,truth=false,prior=2.0', 'dbsnp_138.b37.vcf'),
    ],
    reference_path=variant_discovery.raw_data_container_paths_map[ExtractGenomicVariants.REFERENCE_DIR],
    dry_run=False
)


# Combine sample GVCFs and extract filtered variants for analysis
filtered_vcf_save_path = curated_genotyped_dataset.run_pipeline()


2022-05-26 00:27:12.521 | INFO     | src.gatk.base:_log_operation_header:62 - --------------------
2022-05-26 00:27:12.522 | INFO     | src.gatk.base:_log_operation_header:63 - Stage: Creating IndexFeatureFile for Known Sites
2022-05-26 00:27:12.522 | INFO     | src.gatk.base:_log_operation_header:64 - --------------------
2022-05-26 00:27:12.524 | INFO     | src.gatk.base:_run_command_if_file_not_exists:46 - File /data/sample_gvcfs/human/HG00731.g.vcf.idx already exists; skipping creation.
2022-05-26 00:27:12.525 | INFO     | src.gatk.base:_run_command_if_file_not_exists:46 - File /data/sample_gvcfs/human/HG00732.g.vcf.idx already exists; skipping creation.
2022-05-26 00:27:12.526 | INFO     | src.gatk.base:_run_command_if_file_not_exists:46 - File /data/sample_gvcfs/human/HG00733.g.vcf.idx already exists; skipping creation.
2022-05-26 00:27:12.527 | INFO     | src.gatk.base:download_uris:184 - Skipping download of URI ftp://gsapubftp-anonymous@ftp.broadinstitute.org/bundle/b37/hapmap

/data/Genomics/known_sites/human


In [7]:
mt = vcf_path_to_mt(filtered_vcf_save_path)

Initializing Hail with default parameters...


2022-05-25 23:41:53 WARN  NativeCodeLoader:60 - Unable to load native-hadoop library for your platform... using builtin-java classes where applicable


Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).


2022-05-25 23:41:53 WARN  Hail:43 - This Hail JAR was compiled for Spark 3.1.2, running with Spark 3.1.3.
  Compatibility is not guaranteed.
2022-05-25 23:41:53 WARN  Utils:69 - Service 'SparkUI' could not bind on port 4040. Attempting port 4041.


Running on Apache Spark version 3.1.3
SparkUI available at http://metamorphesis:4041
Welcome to
     __  __     <>__
    / /_/ /__  __/ /
   / __  / _ `/ / /
  /_/ /_/\_,_/_/_/   version 0.2.95-513139587f57
LOGGING: writing to /home/jon/PycharmProjects/reads-to-analysis-ready-variants/examples/hail-20220525-2341-0.2.95-513139587f57.log


# Show the Genotype variant call data 

Notation:
- Reference is alleles[0]
- Variant is alleles[1]
- 0/0 : homozygous reference or aa
- 0/1 : heterozygous or Aa
- 1/1 : homozygous alternate or AA


In [8]:
mt.GT.show(n_rows=100)

,,,,
,,'HG00731','HG00732','HG00733'
locus,alleles,GT,GT,GT
locus<GRCh37>,array<str>,call,call,call
20:65900,"[""G"",""A""]",1/1,1/1,1/1
20:66370,"[""G"",""A""]",1/1,1/1,1/1
20:67500,"[""T"",""TTGGTATCTAG""]",1/1,1/1,1/1
20:68749,"[""T"",""C""]",0/1,0/1,1/1
20:69094,"[""G"",""A""]",0/1,0/1,0/1
20:69481,"[""CT"",""C""]",0/0,0/1,0/0
20:69506,"[""G"",""GACACAC"",""GACAC""]",0/1,1/2,0/1


# Filter missing rows

In [13]:

dataset = mt.filter_rows(hl.agg.any(hl.is_missing(mt.GT)) == False)
print(f"Num NA rows dropped: {mt.count()[0] - dataset.count()[0]}")
mt.filter_rows(hl.agg.any(hl.is_missing(mt.GT)) == True).GT.show()

Num NA rows dropped: 3


,,,,
,,'HG00731','HG00732','HG00733'
locus,alleles,GT,GT,GT
locus<GRCh37>,array<str>,call,call,call
20:37584126,"[""AAAG"",""A""]",NA,0/0,1/1
20:44386269,"[""GTGTATATATATGCATATGTATGTATATATATC"",""G""]",0/0,NA,1/1
20:50252339,"[""A"",""AGGAG""]",0/0,1|1,NA


# Find known_sites where parents are both homozygous reference


In [14]:
parents_hom_ref = mt.filter_rows((hl.literal([True, True]) == hl.agg.collect(mt.GT.is_hom_ref())[0:2]) == True)

parents_hom_ref.GT.show()
print(f"Percentage of total: {round(parents_hom_ref.count()[0]/mt.count()[0] * 100, 2)}%")

,,,,
,,'HG00731','HG00732','HG00733'
locus,alleles,GT,GT,GT
locus<GRCh37>,array<str>,call,call,call
20:114916,"[""G"",""A""]",0/0,0/0,1/1
20:142262,"[""C"",""G""]",0/0,0/0,1|1
20:142339,"[""T"",""C""]",0/0,0/0,1/1
20:221118,"[""G"",""GAAA""]",0/0,0/0,0/1
20:248667,"[""G"",""A""]",0/0,0/0,0/1
20:254517,"[""C"",""CAAGAA""]",0/0,0/0,0|1
20:259563,"[""C"",""A""]",0/0,0/0,1/1


Percentage of total: 1.29%


# Find known_sites where parents are both heterozygous


In [15]:
parents_het = mt.filter_rows((hl.literal([True, True]) == hl.agg.collect(mt.GT.is_het())[0:2]) == True)

parents_het.GT.show()
print(f"Percentage of total: {round(parents_het.count()[0]/mt.count()[0] * 100, 2)}%")

,,,,
,,'HG00731','HG00732','HG00733'
locus,alleles,GT,GT,GT
locus<GRCh37>,array<str>,call,call,call
20:68749,"[""T"",""C""]",0/1,0/1,1/1
20:69094,"[""G"",""A""]",0/1,0/1,0/1
20:69506,"[""G"",""GACACAC"",""GACAC""]",0/1,1/2,0/1
20:76962,"[""T"",""C""]",0/1,0/1,0/0
20:77965,"[""G"",""GT"",""GTT""]",0/1,0/2,0/0
20:82012,"[""T"",""C""]",0/1,0/1,0/0
20:87416,"[""A"",""C""]",0/1,0/1,0/1


Percentage of total: 17.52%


# Sites where child is homozygous variant 

In [16]:

child_hom_var = mt.filter_rows((hl.agg.collect(mt.GT.is_hom_var())[2]==True) == True)

child_hom_var.GT.show()
print(f"Percentage of total: {round(child_hom_var.count()[0]/mt.count()[0] * 100, 2)}%")

,,,,
,,'HG00731','HG00732','HG00733'
locus,alleles,GT,GT,GT
locus<GRCh37>,array<str>,call,call,call
20:65900,"[""G"",""A""]",1/1,1/1,1/1
20:66370,"[""G"",""A""]",1/1,1/1,1/1
20:67500,"[""T"",""TTGGTATCTAG""]",1/1,1/1,1/1
20:68749,"[""T"",""C""]",0/1,0/1,1/1
20:80655,"[""A"",""G""]",1/1,1/1,1/1
20:106703,"[""C"",""CT"",""CTTT""]",0/1,1/2,1/1
20:114916,"[""G"",""A""]",0/0,0/0,1/1


Percentage of total: 27.36%
